In [1]:
import pandas as pd

In [ ]:
# Fungsi keanggotaan 
def triangular(x, a, b, c):
    if x <= a or x >= c:
        return 0.0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x < c:
        return (c - x) / (c - b)
    else:
        return 0.0

In [ ]:
# Mengkategorikan data keanggotaan fuzzy (nilai linguistik)
def glucose_membership(x):
    return {
        'rendah': triangular(x, 0, 0, 0.5),
        'sedang': triangular(x, 0, 0.5, 1),
        'tinggi': triangular(x, 0.5, 1, 1)
    }
def bmi_membership(x):
    return {
        'rendah': triangular(x, 0, 0, 0.5),
        'sedang': triangular(x, 0, 0.5, 1),
        'tinggi': triangular(x, 0.5, 1, 1)
    }

def age_membership(x):
    return {
        'muda': triangular(x, 0, 0, 0.5),
        'paruh_baya': triangular(x, 0, 0.5, 1),
        'tua': triangular(x, 0.5, 1, 1)
    }
def dpf_membership(x):
    return {
        'rendah': triangular(x, 0, 0, 0.5),
        'sedang': triangular(x, 0, 0.5, 1),
        'tinggi': triangular(x, 0.5, 1, 1)
    }

def bp_membership(x):
    return {
        'rendah': triangular(x, 0, 0, 0.5),
        'sedang': triangular(x, 0, 0.5, 1),
        'tinggi': triangular(x, 0.5, 1, 1)
    }

In [ ]:
def sugeno_inference(glucose, bmi, age, dpf, bp):
    g = glucose_membership(glucose)
    b = bmi_membership(bmi)
    a = age_membership(age)
    d = dpf_membership(dpf)
    p = bp_membership(bp)
    # Fuzzy rule
    rules = [
        # Rule 1: Semua tinggi -> risiko tinggi (0.9)
        (min(g['tinggi'], b['tinggi'], a['tua'], d['tinggi'], p['tinggi']), 0.9),

        # Rule 2: Semua sedang -> risiko sedang (0.5)
        (min(g['sedang'], b['sedang'], a['paruh_baya'], d['sedang'], p['sedang']), 0.5),

        # Rule 3: Semua rendah -> risiko rendah (0.1)
        (min(g['rendah'], b['rendah'], a['muda'], d['rendah'], p['rendah']), 0.1),

        # Rule 4: Glukosa tinggi meskipun faktor lain sedang -> risiko tinggi (0.8)
        (min(g['tinggi'], b['sedang'], a['paruh_baya'], d['sedang'], p['sedang']), 0.8),

        # Rule 5: BMI sangat tinggi (obesitas) dan Usia tua, meskipun Glukosa masih sedang -> risiko agak tinggi (0.7)
        (min(g['sedang'], b['tinggi'], a['tua']), 0.7),

        # Rule 6: Diabetes Pedigree Function (faktor keturunan) tinggi, dan usia sudah tua, glukosa sedang -> risiko agak tinggi (0.75)
        (min(g['sedang'], d['tinggi'], a['tua']), 0.75),

        # Rule 7: Tekanan Darah (BloodPressure) tinggi DAN BMI tinggi, meskipun glukosa sedang -> risiko sedang ke tinggi (0.65)
        (min(g['sedang'], b['tinggi'], p['tinggi']), 0.65),

        # Rule 8: Usia muda, Glukosa dan BMI normal/rendah, tapi DPF tinggi (faktor keturunan kuat) -> risiko sedikit meningkat/waspada (0.3)
        (min(g['rendah'], b['rendah'], a['muda'], d['tinggi']), 0.3),

        # Rule 9: Semua faktor rendah atau normal, kecuali tekanan darah sedikit tinggi -> risiko rendah (0.2)
        (min(g['rendah'], b['rendah'], a['muda'], d['rendah'], p['sedang']), 0.2),

        # Rule 10: Glukosa sedang, BMI sedang, Usia paruh baya, TAPI DPF tinggi dan BP tinggi -> risiko tinggi (0.85)
        (min(g['sedang'], b['sedang'], a['paruh_baya'], d['tinggi'], p['tinggi']), 0.85),

        # Rule 11: Glukosa tinggi, yang lain rendah/sedang -> tetap risiko tinggi karena glukosa (0.8)
        (g['tinggi'], 0.8),

        # Rule 12: Walaupun usia muda dan BMI normal, jika glukosa sudah tinggi -> risiko sedang ke tinggi (0.6)
        (min(g['tinggi'], b['rendah'], a['muda']), 0.6),
    ]

    numerator = sum(alpha * z for alpha, z in rules)
    denominator = sum(alpha for alpha, _ in rules)
    if denominator == 0:
        return 0
    return numerator / denominator

    return numerator / denominator

In [11]:
# Load data yang telah dipreprocessing
df = pd.read_csv('diabetes_preprocessed.csv')

In [13]:
# Mengambil fitur yang dipakai
fitur = ['Glucose', 'BMI', 'Age', 'DiabetesPedigreeFunction', 'BloodPressure']

In [ ]:
for idx, row in df[fitur].head().iterrows():
    hasil = sugeno_inference(
        row['Glucose'],
        row['BMI'],
        row['Age'],
        row['DiabetesPedigreeFunction'],
        row['BloodPressure']
    )
    print(f"Data ke {idx} - Sugeno Risk Score: {hasil:.3f}")

Index 0 - Sugeno Risk Score: 0.655
Index 1 - Sugeno Risk Score: 0.311
Index 2 - Sugeno Risk Score: 0.698
Index 3 - Sugeno Risk Score: 0.000
Index 4 - Sugeno Risk Score: 0.800


Uji Evaluasi Fuzzy System menggunakan metode Sugeno

In [16]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# 1. Hitung skor risiko Sugeno untuk seluruh data
risk_scores = []
for idx, row in df[fitur].iterrows():
    hasil = sugeno_inference(
        row['Glucose'],
        row['BMI'],
        row['Age'],
        row['DiabetesPedigreeFunction'],
        row['BloodPressure']
    )
    risk_scores.append(hasil)

In [ ]:
# 2. Tentukan threshold untuk klasifikasi
threshold = 0.5
prediksi = [1 if score >= threshold else 0 for score in risk_scores]

In [18]:
# 3. Ambil label asli
y_true = df['Outcome'].values

In [19]:
# 4. Hitung metrik evaluasi
akurasi = accuracy_score(y_true, prediksi)
f1 = f1_score(y_true, prediksi)
precision = precision_score(y_true, prediksi)
recall = recall_score(y_true, prediksi)

print(f"Akurasi Sugeno: {akurasi:.3f}")
print(f"F1-score Sugeno: {f1:.3f}")
print(f"Precision Sugeno: {precision:.3f}")
print(f"Recall Sugeno: {recall:.3f}")

Akurasi Sugeno: 0.674
F1-score Sugeno: 0.626
Precision Sugeno: 0.522
Recall Sugeno: 0.780
